### King County Request | December 2022

"Total onroad vehicle emissions and activity data within each jurisdiction's geographic boundary for the calendar years 2019 and 2020. If available, we would also request projections over time going to 2050. Only aggregate data is needed, broken down by the vehicle mode and fuel type, as shown in the table. If there is data that doesn't fit into the listed categories, please let us know."

Requested for all King County cities, King County total and King County unicorporated areas.

Years required include 2018, 2019, 2020, 2030, 2040, 2050. Interpolation between 2018 and 2030 are used to provide 2019 and 2020 estimates. 


A script was created to address the data request, which allows standard Soundcast network outputs to be segmented to the jurisdiction level. The "aq_tool" notebook processes Soundcast output for a list of provided cities and reports emissions and VMT by light, medium, and heavy vehicles. 

This notebook compiles and organizes the results of the aq_tool.

In [239]:
import os, sys
import pandas as pd
sys.path.append('../.')
import emissions

In [11]:
# Location of output from aq_tool
output_dir = r'C:\Workspace\travel-modeling\aq_tool\output\HOLD\king'

In [195]:
year = '2018'
df_18 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

year = '2030'
df_30 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')
df_30.rename(columns={'total_daily_tons': 'total_daily_tons_2030',
                      'vmt': 'vmt_2030'}, inplace=True)

year = '2040'
df_40 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')
df_40.rename(columns={'total_daily_tons': 'total_daily_tons_2040',
                      'vmt': 'vmt_2040'}, inplace=True)

year = '2050'
df_50 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')
df_50.rename(columns={'total_daily_tons': 'total_daily_tons_2050',
                      'vmt': 'vmt_2050'}, inplace=True)

In [236]:
run_dir_2018 = r'L:\RTP_2022\final_runs\sc_rtp_2018_final\soundcast'
run_dir_2030 = r'L:\RTP_2022\final_runs\sc_rtp_2030_final\soundcast'
run_dir_2040 = r'L:\RTP_2022\final_runs\sc_rtp_2040_final\soundcast'
run_dir_2050 = r'L:\RTP_2022\final_runs\sc_rtp_2050_constrained_final_revised\soundcast'

In [119]:
# For 2019 - 2023, use interpolated county totals and scale to the city level

# The county script (create_county_emissions_interpolated.py) generates 
# interpolated county emissions totals by non-model years by creating new emissions rates
# interpolated from 2018 and 2030 year ends. All roadway volumes are scaled by HPMS changes
# Total emissions are calculated based on these revised  rates and volumes. 

# For cities, we will take these new county totals and scale the overall changes 
# at the city level. For instance if there are a total of 100 tons of emissions
# for a city in 2018 and the county emissions show an increase of 10%, we
# will scale the city emissions for that year by 10%. This way we keep the
# model year distribution of VMT within city boundaries but accoutn for changes in 
# county-level VMT and fleet changes. 

# Load County totals
# df_18_county = pd.read_csv(r'C:\Workspace\travel-modeling\aq_tool\output\interpolated\King\2018\running_summary.csv')

## Process City Emissions
king_county_cities_emissions.csv

In [209]:
veh_type_map = {'heavy': 'heavy',
                    'medium': 'medium',
                    'sov': 'light',
                    'hov2': 'light',
                    'hov3': 'light'}

In [205]:
def create_county_total(pollutant_name):

    county_total = pd.DataFrame()

    for year in ['2018','2019','2020','2021','2022','2023']:

        df_county = pd.read_csv(os.path.join(r'C:\Workspace\travel-modeling\aq_tool\output\interpolated\King',
                                            str(year), 
                                            'running_summary.csv'))

        # Scale by light, medium, heavy
        df = df_county[df_county['pollutant_name']==pollutant_name][['veh_type','running_daily_tons']]
        df['veh_cat'] = df['veh_type'].map(veh_type_map)
        df = df.groupby('veh_cat').sum()[['running_daily_tons']].reset_index()
        df.rename(columns={'running_daily_tons':pollutant_name}, inplace=True)
        df['year'] = year
        county_total = county_total.append(df)

    return county_total

In [202]:
def scale_emissions(year_df, pollutant_name):

    county_total = create_county_total(pollutant_name)
    df = year_df[year_df['pollutant_name'] == pollutant_name]

    # Join the county totals to the 2018 numbers to scale to 2019, etc.
    # county_total

    # Group so that light medium and heavy are rows and each year has a scaling factor
    factors = pd.pivot_table(county_total, index='veh_cat', columns='year', values=pollutant_name,
                aggfunc='sum')
    year_list = ['2019','2020','2021','2022','2023']
    for year in year_list:
        factors[year+'_factor'] = factors[year]/factors['2018']

    factors = factors[[year+'_factor' for year in year_list]].reset_index()

    df = df.merge(factors, how='left', left_on='veh_type', right_on='veh_cat')
    for year in year_list:
        df['total_daily_tons_'+year] = df[year+'_factor']*df['total_daily_tons']

    df.rename(columns={'total_daily_tons': 'total_daily_tons_2018'}, inplace=True)

    df = df[['city','pollutant_name','veh_type','total_daily_tons_2018','total_daily_tons_2019',
        'total_daily_tons_2020','total_daily_tons_2021','total_daily_tons_2022','total_daily_tons_2023']]

    return df

In [203]:
df_co2e = scale_emissions(df_18,'CO2 Equivalent')
df_co2_atmo = scale_emissions(df_18,'Atmospheric CO2')
df = pd.concat([df_co2e, df_co2_atmo])

df = df.sort_values(['city','pollutant_name','veh_type'])

# Add in additional model years from existing summaries
df = df.merge(df_30, on=['city','veh_type','pollutant_name'], how='left')
df.drop('vmt_2030', axis=1, inplace=True)
df = df.merge(df_40, on=['city','veh_type','pollutant_name'], how='left')
df.drop('vmt_2040', axis=1, inplace=True)
df = df.merge(df_50, on=['city','veh_type','pollutant_name'], how='left')
df.drop('vmt_2050', axis=1, inplace=True)

df.to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\king_county_cities_emissions.csv')


# Process VMT
king_county_cities_vmt.csv

In [204]:
def scale_vmt(year_df, pollutant_name):

    county_total = create_county_total('CO2 Equivalent')

    df = year_df[year_df['pollutant_name'] == pollutant_name]

    # Join the county totals to the 2018 numbers to scale to 2019, etc.
    # county_total

    # Group so that light medium and heavy are rows and each year has a scaling factor
    factors = pd.pivot_table(county_total, index='veh_cat', columns='year', values='daily_vmt_total',
                aggfunc='sum')
    year_list = ['2019','2020','2021','2022','2023']
    for year in year_list:
        factors[year+'_factor'] = factors[year]/factors['2018']

    factors = factors[[year+'_factor' for year in year_list]].reset_index()

    df = df.merge(factors, how='left', left_on='veh_type', right_on='veh_cat')
    for year in year_list:
        df['daily_vmt_total_'+year] = df[year+'_factor']*df['vmt']

    df.rename(columns={'vmt': 'daily_vmt_total_2018'}, inplace=True)

    df = df[['city','pollutant_name','veh_type','daily_vmt_total_2018','daily_vmt_total_2019',
        'daily_vmt_total_2020','daily_vmt_total_2021','daily_vmt_total_2022','daily_vmt_total_2023']]

    return df

In [197]:
df_vmt = scale_vmt(df_18,'CO2 Equivalent')
df = df_vmt.sort_values(['city','pollutant_name','veh_type'])

_df_30 = df_30[df_30['pollutant_name'] == 'CO2 Equivalent']
_df_40 = df_40[df_40['pollutant_name'] == 'CO2 Equivalent']
_df_50 = df_50[df_50['pollutant_name'] == 'CO2 Equivalent']

# Add in additional model years from existing summaries
df = df.merge(_df_30[['city','veh_type','vmt_2030']], on=['city','veh_type'], how='left')
df.drop('pollutant_name', axis=1, inplace=True)
df.rename(columns={'vmt_2030': 'daily_vmt_total_2030'}, inplace=True)

df = df.merge(_df_40[['city','veh_type','vmt_2040']], on=['city','veh_type'], how='left')
# df.drop('pollutant_name', axis=1, inplace=True)
df.rename(columns={'vmt_2040': 'daily_vmt_total_2040'}, inplace=True)

df = df.merge(_df_50[['city','veh_type','vmt_2050']], on=['city','veh_type'], how='left')
# df.drop('pollutant_name', axis=1, inplace=True)
df.rename(columns={'vmt_2050': 'daily_vmt_total_2050'}, inplace=True)

df.to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\king_county_cities_vmt.csv')


## Regional Emissions
regional_emissions_by_county.csv

In [233]:
def get_county_emissions(run_dir, county=None):
    
    # Get county level totals
    df_interzonal = pd.read_csv(os.path.join(run_dir,r'outputs\emissions\interzonal_emissions.csv'))
    df_intrazonal = pd.read_csv(os.path.join(run_dir,r'outputs\emissions\intrazonal_emissions.csv'))
    start_emissions_df = pd.read_csv(os.path.join(run_dir, r'outputs\emissions\start_emissions.csv'))
    df_inter_group = df_interzonal.groupby(['pollutantID','veh_type','county']).sum()[['tons_tot','vmt']].reset_index()
    df_inter_group.rename(columns={'tons_tot': 'interzonal_tons'}, inplace=True)
    df_intra_group = df_intrazonal.groupby(['pollutantID','veh_type','county']).sum()[['tons_tot','vmt']].reset_index()
    df_intra_group.rename(columns={'tons_tot': 'intrazonal_tons'}, inplace=True)
    df_start_group = start_emissions_df.groupby(['pollutantID','veh_type','county']).sum()[['start_tons']].reset_index()

    from emissions import finalize_emissions, pollutant_map
    summary_df = pd.merge(df_inter_group, df_intra_group, on=['veh_type','pollutantID','county'], suffixes=['_interzonal','_intrazonal'])
    summary_df['vmt'] = summary_df['vmt_interzonal']+summary_df['vmt_intrazonal']
    summary_df = pd.merge(summary_df, df_start_group, how='left', on=['veh_type','pollutantID','county'])
    # summary_df = finalize_emissions(summary_df, col_suffix="")
    pm10 = summary_df[summary_df['pollutantID'].isin([100,106,107])].groupby(['veh_type','county']).sum().reset_index()
    pm10['pollutantID'] = 'PM10'
    pm25 = summary_df[summary_df['pollutantID'].isin([110,116,117])].groupby(['veh_type','county']).sum().reset_index()
    pm25['pollutantID'] = 'PM25'
    summary_df = summary_df.append(pm10)
    summary_df = summary_df.append(pm25)
    summary_df.loc[~summary_df['pollutantID'].isin(['PM','PM10','PM25']),'pollutantID'] = summary_df[~summary_df['pollutantID'].isin(['PM','PM10','PM25'])]['pollutantID'].astype('int')
    summary_df['pollutant_name'] = summary_df['pollutantID'].astype('int', errors='ignore').astype('str').map(pollutant_map)
    summary_df['total_daily_tons'] = summary_df['start_tons']+summary_df['interzonal_tons']+summary_df['intrazonal_tons']
    summary_df = summary_df[['pollutantID','county','pollutant_name','veh_type','start_tons','intrazonal_tons','interzonal_tons','total_daily_tons','vmt']]

    # Filter for county
    if county:
        summary_df = summary_df[summary_df['county'] == county]

    return summary_df

In [266]:
county_df = pd.DataFrame()
for county in ['king','kitsap','pierce','snohomish']:    
    # run_dir = r'\\modelstation2\c$\Workspace\sc_2018_rtp_final\soundcast'
    df_18_county = get_county_emissions(run_dir_2018, county)

    # run_dir = r'\\modelstation1\c$\workspace\sc_rtp_2030_final\soundcast'
    df_30_county = get_county_emissions(run_dir_2030, county)

    # run_dir = r'\\modelstation1\c$\workspace\sc_2040_rtp_final\soundcast'
    df_40_county = get_county_emissions(run_dir_2040, county)

    # run_dir = r'L:\RTP_2022\final_runs\sc_rtp_2050_constrained_final\soundcast'
    df_50_county = get_county_emissions(run_dir_2050, county)


    # Create interpolations
    df = df_18_county.merge(df_30_county, on=['pollutant_name','veh_type'], suffixes=['_2018','_2030'], how='outer')

    # Merge 2040
    year = '2040'
    df = df.merge(df_40_county, on=['pollutant_name','veh_type'], how='outer')
    df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                        'vmt': 'vmt_'+year}, inplace=True)

    # # Merge 2050
    year = '2050'
    df = df.merge(df_50_county, on=['pollutant_name','veh_type'], how='outer')
    df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                        'vmt': 'vmt_'+year}, inplace=True)


    # df_vmt = interpolate(df, 2018, 2030, 2019, 'vmt')
    # df_vmt = interpolate(df, 2018, 2030, 2020, 'vmt')
    # df_vmt = df_vmt[['pollutant_name','veh_type','vmt_2018','vmt_2019','vmt_2020','vmt_2030','vmt_2040','vmt_2050']]

    # df_emissions = interpolate(df, 2018, 2030, 2019, 'total_daily_tons')
    # df_emissions = interpolate(df, 2018, 2030, 2020, 'total_daily_tons')
    # df_emissions = df_emissions[['pollutant_name','veh_type','total_daily_tons_2018', 'total_daily_tons_2019',
    #     'total_daily_tons_2020','total_daily_tons_2030','total_daily_tons_2040','total_daily_tons_2050']]

    df = df[df['pollutant_name'].isin(['Atmospheric CO2','CO2 Equivalent','Methane','N20'])]
    df = df[['county_2018','pollutant_name','total_daily_tons_2018','total_daily_tons_2030','total_daily_tons_2040','total_daily_tons_2050',
             'vmt_2018','vmt_2030','vmt_2040','vmt_2050']]
    df['county'] = county

    county_df = pd.concat([county_df, df])
    # # df_emissions.to_csv(r'C:\Workspace\aq_tool\output\\'+county+'_county_total_emissions.csv', index=False)
    # df_emissions.to_csv(os.path.join(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24',
    #                             county+'_county_total_emissions.csv', index=False)
    #                             )

    # _df_vmt = df_vmt.groupby('veh_type').first().reset_index()
    # _df_vmt['geography'] = county.capitalize()+' County Total'
    # _df_vmt.drop('pollutant_name', axis=1, inplace=True)
    # _df_vmt.to_csv(os.path.join(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24',
    #                             county+'_county_total_vmt.csv', index=False)
    #                             )


In [268]:
county_df.head()

,county_2018,pollutant_name,total_daily_tons_2018,total_daily_tons_2030,total_daily_tons_2040,total_daily_tons_2050,vmt_2018,vmt_2030,vmt_2040,vmt_2050,county
9,king,Methane,0.110924,0.286947,0.360990,0.418009,1.462328e+06,1.680054e+06,1.884896e+06,2.076454e+06,king
10,king,Methane,1.549000,0.873218,0.702506,0.694832,4.566289e+07,4.536422e+07,4.763493e+07,5.006812e+07,king
11,king,Methane,0.404293,0.446480,0.469226,0.498593,2.094469e+06,2.255105e+06,2.315409e+06,2.307600e+06,king
12,king,N20,0.003816,0.004879,0.005717,0.006507,1.462328e+06,1.680054e+06,1.884896e+06,2.076454e+06,king
13,king,N20,0.431183,0.266306,0.271915,0.293538,4.566289e+07,4.536422e+07,4.763493e+07,5.006812e+07,king


In [269]:
# NOTE: the emissions totals for cities in a county won't equal the total for a county because of unincorporated emissions
# For county totals we could re-run the interpolation or just use the average/total changes for a county
df_result = pd.DataFrame()

for year in ['2018','2019','2020','2021','2022','2023']:
    
    df = pd.read_csv(os.path.join(r'C:\Workspace\travel-modeling\aq_tool\output\interpolated\King',
                                            str(year), 
                                            'running_summary.csv'))
    df['veh_cat'] = df['veh_type'].map(veh_type_map)
    df = df.groupby(['pollutant_name','veh_cat']).sum()[['daily_vmt_total','running_daily_tons']].reset_index()

    # Get start emissions
    df_start = pd.read_csv(os.path.join(r'C:\Workspace\travel-modeling\aq_tool\output\interpolated\King',
                                                str(year), 
                                                'start_summary.csv'))
    df = df.merge(df_start, left_on=['veh_cat','pollutant_name'], right_on=['veh_type','pollutant_name'], 
            how='left')
    df['total_daily_tons'] = df['start_tons']+df['running_daily_tons']
    df['year'] = year

    df = df[df['pollutant_name'].isin(['Methane', 'N20', 'Atmospheric CO2', 'CO2 Equivalent'])]

    df_result = df_result.append(df[['year','veh_cat','pollutant_name','total_daily_tons','daily_vmt_total']])

tons_sum = pd.pivot_table(df_result, index=['pollutant_name','veh_cat'], columns='year', values='total_daily_tons', aggfunc='sum').reset_index()
vmt_sum = pd.pivot_table(df_result, index=['pollutant_name','veh_cat'], columns='year', values='total_daily_tons', aggfunc='sum').reset_index()

In [274]:
tons_sum

year,pollutant_name,veh_cat,2018,2019,2020,2021,2022,2023
0,Atmospheric CO2,heavy,2604.211970,2569.804535,2091.469230,2210.562939,2217.261691,2238.306450
1,Atmospheric CO2,light,17305.916142,16979.527196,13920.287940,14552.793377,14489.779335,14509.218541
2,Atmospheric CO2,medium,1398.294912,1377.891486,1146.369039,1199.521633,1199.056131,1205.145919
3,CO2 Equivalent,heavy,2607.776080,2573.656189,2094.918931,2214.499451,2221.520171,2242.925037
4,CO2 Equivalent,light,17464.046175,17133.122283,14060.785935,14691.679871,14624.768630,14640.412820
5,CO2 Equivalent,medium,1437.445844,1416.350632,1183.207074,1235.921245,1234.761153,1240.157609
6,Methane,heavy,0.101003,0.112000,0.102197,0.118899,0.130698,0.143761
7,Methane,light,1.443566,1.396180,1.245558,1.234825,1.196769,1.160249
8,Methane,medium,0.394738,0.397694,0.383179,0.391438,0.395318,0.399855
9,N20,heavy,0.003488,0.003530,0.003003,0.003235,0.003325,0.003437


In [231]:
# Merge data from older data requests
tons_sum

year,pollutant_name,veh_cat,2018,2019,2020,2021,2022,2023
0,Atmospheric CO2,heavy,2604.211970,2569.804535,2091.469230,2210.562939,2217.261691,2238.306450
1,Atmospheric CO2,light,17305.916142,16979.527196,13920.287940,14552.793377,14489.779335,14509.218541
2,Atmospheric CO2,medium,1398.294912,1377.891486,1146.369039,1199.521633,1199.056131,1205.145919
3,CO2 Equivalent,heavy,2607.776080,2573.656189,2094.918931,2214.499451,2221.520171,2242.925037
4,CO2 Equivalent,light,17464.046175,17133.122283,14060.785935,14691.679871,14624.768630,14640.412820
5,CO2 Equivalent,medium,1437.445844,1416.350632,1183.207074,1235.921245,1234.761153,1240.157609
6,Methane,heavy,0.101003,0.112000,0.102197,0.118899,0.130698,0.143761
7,Methane,light,1.443566,1.396180,1.245558,1.234825,1.196769,1.160249
8,Methane,medium,0.394738,0.397694,0.383179,0.391438,0.395318,0.399855
9,N20,heavy,0.003488,0.003530,0.003003,0.003235,0.003325,0.003437


## king_county_unincorporated_vmt.csv

In [ ]:
# Calculate as sum of county totals minus city boundaries




In [13]:
# Create interpolations
df = df_18.merge(df_30, on=['pollutant_name','city','veh_type'], suffixes=['_2018','_2030'], how='outer')

# Merge 2040
year = '2040'
df = df.merge(df_40, on=['pollutant_name','city','veh_type'], how='outer')
df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                    'vmt': 'vmt_'+year}, inplace=True)

# Merge 2050
year = '2050'
df = df.merge(df_50, on=['pollutant_name','city','veh_type'], how='outer')
df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                    'vmt': 'vmt_'+year}, inplace=True)


# df.head()

In [14]:
def interpolate(df, start_year, end_year, interpolated_year, var):
    """Interpolate values from two cols of data given start, end, and target (interpolated) years."""
    
    # Calculate annual rate of change based on difference between values at start and end year, divided number of years difference
    df['annual_'+var+'_change'] = (df[var+'_'+str(end_year)] - df[var+'_'+str(start_year)])/(end_year-start_year)
    # Calculate 
    df[var+'_'+str(interpolated_year)] = df[var+'_'+str(start_year)]+df['annual_'+var+'_change']*(interpolated_year-start_year)

    return df                                 

In [15]:
df_vmt = interpolate(df, 2018, 2030, 2019, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2020, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2021, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2022, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2023, 'vmt')


In [19]:
# df_vmt

In [17]:
df_vmt = df_vmt[['city','pollutant_name','veh_type','vmt_2018','vmt_2019','vmt_2020',
                 'vmt_2021','vmt_2022','vmt_2023','vmt_2030','vmt_2040','vmt_2050']]

# VMT is duplicated for each pollutant total; Select only the first set of rows
df_vmt = df_vmt.drop('pollutant_name', axis=1)
df_vmt = df_vmt.groupby(['city','veh_type']).first()

df_vmt.to_csv(r'output\HOLD\king_county_cities_vmt.csv')

KeyError: "['pollutant_name', 'veh_type', 'city'] not in index"

In [57]:
# Get county totals by vehicle type
df_city_tot_vmt = df_vmt.groupby('veh_type').sum().reset_index()

#### Compile Emissions

In [58]:
df_emissions = interpolate(df, 2018, 2030, 2019, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2020, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2021, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2022, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2023, 'total_daily_tons')

In [59]:
df_emissions = df_emissions[['city','pollutant_name','veh_type','total_daily_tons_2018', 'total_daily_tons_2019',
    'total_daily_tons_2020','total_daily_tons_2021','total_daily_tons_2022','total_daily_tons_2023',
    'total_daily_tons_2030','total_daily_tons_2040','total_daily_tons_2050']]
df_emissions = df_emissions[df_emissions['pollutant_name'].isin(['Atmospheric CO2','CO2 Equivalent'])]

In [60]:
df_emissions.to_csv(r'C:\Workspace\aq_tool\output\king_county_cities_emissions.csv', index=False)

In [61]:
# Get county totals by vehicle type
df_city_tot_emissions = df_emissions.drop('city', axis=1).groupby(['pollutant_name','veh_type']).sum().reset_index()

### Compile Totals for County and Unincorporated Areas

Get county level results from the outputs of the regional results

In [15]:
def get_county_emissions(run_dir, county=None):
    
    # Get county level totals
    df_interzonal = pd.read_csv(os.path.join(run_dir,r'outputs\emissions\interzonal_emissions.csv'))
    df_intrazonal = pd.read_csv(os.path.join(run_dir,r'outputs\emissions\intrazonal_emissions.csv'))
    start_emissions_df = pd.read_csv(os.path.join(run_dir, r'outputs\emissions\start_emissions.csv'))
    df_inter_group = df_interzonal.groupby(['pollutantID','veh_type','county']).sum()[['tons_tot','vmt']].reset_index()
    df_inter_group.rename(columns={'tons_tot': 'interzonal_tons'}, inplace=True)
    df_intra_group = df_intrazonal.groupby(['pollutantID','veh_type','county']).sum()[['tons_tot','vmt']].reset_index()
    df_intra_group.rename(columns={'tons_tot': 'intrazonal_tons'}, inplace=True)
    df_start_group = start_emissions_df.groupby(['pollutantID','veh_type','county']).sum()[['start_tons']].reset_index()

    from emissions import finalize_emissions, pollutant_map
    summary_df = pd.merge(df_inter_group, df_intra_group, on=['veh_type','pollutantID','county'], suffixes=['_interzonal','_intrazonal'])
    summary_df['vmt'] = summary_df['vmt_interzonal']+summary_df['vmt_intrazonal']
    summary_df = pd.merge(summary_df, df_start_group, how='left', on=['veh_type','pollutantID','county'])
    # summary_df = finalize_emissions(summary_df, col_suffix="")
    pm10 = summary_df[summary_df['pollutantID'].isin([100,106,107])].groupby(['veh_type','county']).sum().reset_index()
    pm10['pollutantID'] = 'PM10'
    pm25 = summary_df[summary_df['pollutantID'].isin([110,116,117])].groupby(['veh_type','county']).sum().reset_index()
    pm25['pollutantID'] = 'PM25'
    summary_df = summary_df.append(pm10)
    summary_df = summary_df.append(pm25)
    summary_df.loc[~summary_df['pollutantID'].isin(['PM','PM10','PM25']),'pollutantID'] = summary_df[~summary_df['pollutantID'].isin(['PM','PM10','PM25'])]['pollutantID'].astype('int')
    summary_df['pollutant_name'] = summary_df['pollutantID'].astype('int', errors='ignore').astype('str').map(pollutant_map)
    summary_df['total_daily_tons'] = summary_df['start_tons']+summary_df['interzonal_tons']+summary_df['intrazonal_tons']
    summary_df = summary_df[['pollutantID','county','pollutant_name','veh_type','start_tons','intrazonal_tons','interzonal_tons','total_daily_tons','vmt']]

    # Filter for county
    if county:
        summary_df = summary_df[summary_df['county'] == county]

    return summary_df

In [44]:
for county in ['king','kitsap','pierce','snohomish']:    
    run_dir = r'L:\RTP_2022\final_runs\sc_rtp_2018_final\soundcast'
    df_18_county = get_county_emissions(run_dir, county)

    run_dir = r'L:\RTP_2022\final_runs\sc_rtp_2030_final\soundcast'
    df_30_county = get_county_emissions(run_dir, county)

    run_dir = r'L:\RTP_2022\final_runs\sc_rtp_2040_final\soundcast'
    df_40_county = get_county_emissions(run_dir, county)

    run_dir = r'L:\RTP_2022\final_runs\sc_rtp_2050_constrained_final_revised\soundcast'
    df_50_county = get_county_emissions(run_dir, county)


    # Create interpolations
    df = df_18_county.merge(df_30_county, on=['pollutant_name','veh_type'], suffixes=['_2018','_2030'], how='outer')

    # Merge 2040
    year = '2040'
    df = df.merge(df_40_county, on=['pollutant_name','veh_type'], how='outer')
    df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                        'vmt': 'vmt_'+year}, inplace=True)

    # # Merge 2050
    year = '2050'
    df = df.merge(df_50_county, on=['pollutant_name','veh_type'], how='outer')
    df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                        'vmt': 'vmt_'+year}, inplace=True)


    df_vmt = interpolate(df, 2018, 2030, 2019, 'vmt')
    df_vmt = interpolate(df, 2018, 2030, 2020, 'vmt')
    df_vmt = interpolate(df, 2018, 2030, 2021, 'vmt')
    df_vmt = interpolate(df, 2018, 2030, 2022, 'vmt')
    df_vmt = interpolate(df, 2018, 2030, 2023, 'vmt')
    df_vmt = df_vmt[['pollutant_name','veh_type','vmt_2018','vmt_2019','vmt_2020',
                     'vmt_2021','vmt_2022','vmt_2023','vmt_2030','vmt_2040','vmt_2050']]

    df_emissions = interpolate(df, 2018, 2030, 2019, 'total_daily_tons')
    df_emissions = interpolate(df, 2018, 2030, 2020, 'total_daily_tons')
    df_emissions = interpolate(df, 2018, 2030, 2021, 'total_daily_tons')
    df_emissions = interpolate(df, 2018, 2030, 2022, 'total_daily_tons')
    df_emissions = interpolate(df, 2018, 2030, 2023, 'total_daily_tons')
    df_emissions = df_emissions[['pollutant_name','veh_type','total_daily_tons_2018', 'total_daily_tons_2019',
        'total_daily_tons_2020','total_daily_tons_2021','total_daily_tons_2022','total_daily_tons_2023',
        'total_daily_tons_2030','total_daily_tons_2040','total_daily_tons_2050']]

    df_emissions = df_emissions[df_emissions['pollutant_name'].isin(['Atmospheric CO2','CO2 Equivalent','Methane','N20'])]
    df_emissions.to_csv(r'C:\Workspace\aq_tool\output\\'+county+'_county_total_emissions.csv', index=False)

    _df_vmt = df_vmt.groupby('veh_type').first().reset_index()
    _df_vmt['geography'] = county.capitalize()+' County Total'
    _df_vmt.drop('pollutant_name', axis=1, inplace=True)
    _df_vmt.to_csv(r'C:\Workspace\aq_tool\output\\'+county+'_county_total_vmt.csv', index=False)


In [46]:
# Join county totals together
county_summary_df = pd.DataFrame()
county_vmt_df = pd.DataFrame()
for county in ['king','kitsap','pierce','snohomish']:
    df = pd.read_csv(r'C:\Workspace\aq_tool\output\\'+county+'_county_total_emissions.csv')
    df['county'] = county
    county_summary_df = county_summary_df.append(df)

    df = pd.read_csv(r'C:\Workspace\aq_tool\output\\'+county+'_county_total_vmt.csv')
    df['county'] = county
    county_vmt_df = county_vmt_df.append(df)

county_summary_df.to_csv(r'C:\Workspace\aq_tool\output\regional_emissions_by_county.csv', index=False)
county_vmt_df.to_csv(r'C:\Workspace\aq_tool\output\regional_vmt_by_county.csv', index=False)


In [80]:
king_total_df = county_summary_df[county_summary_df['county'] == 'king']
king_vmt_df = county_vmt_df[county_vmt_df['county'] == 'king']
# df_emissions


In [76]:
# df_city_tot_emissions

In [78]:
# 
df = df_city_tot_emissions.merge(king_total_df, suffixes=['_cities_tot','_county_tot'], on=['veh_type','pollutant_name'])
for year in [2018,2019,2020,2021,2022,2023,2030,2040,2050]:
    df['total_daily_tons_unincorporated_'+str(year)] = df['total_daily_tons_'+str(year)+'_county_tot']-df['total_daily_tons_'+str(year)+'_cities_tot']

In [79]:
df[['pollutant_name','veh_type']+['total_daily_tons_unincorporated_'+str(year) for year in ['2018',
    '2019','2020','2021','2022','2023','2030','2040','2050']]].to_csv(r'C:\Workspace\aq_tool\output\king_county_unincorporated_emissions.csv', index=False)

In [81]:
df = king_vmt_df.drop('geography', axis=1).merge(df_city_tot_vmt, suffixes=['_county','_city_total'], on='veh_type')
# df = df_vmt.merge(df_emissions, suffixes=['_cities_tot','_county_tot'], on=['veh_type','pollutant_name'])
for year in [2018,2019,2020,2021,2022,2023,2030,2040,2050]:
    df['vmt_unincorporated_'+str(year)] = df['vmt_'+str(year)+'_county']-df['vmt_'+str(year)+'_city_total']

In [82]:
df['geography'] = 'Unincorporated King'
df[['veh_type','geography']+['vmt_unincorporated_'+str(year) for year in ['2018',
    '2019','2020','2021','2022','2023','2030','2040','2050']]].to_csv(r'C:\Workspace\aq_tool\output\king_county_unincorporated_vmt.csv', index=False)